In [1]:
# #run this to update tensorflow or keras in case the model fails to load...
# import sys
# !{sys.executable} -m pip --upgrade tensorflow
# !{sys.executable} -m pip --upgrade keras

In [2]:
import tensorflow as tf
import numpy as np
import cv2 
import math
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [3]:
def get_model():
    model_path = r"key_model"
    model = tf.keras.models.load_model(model_path)
    return model
    



In [4]:
def key_fit(image , keys, w, h):
    color = (255,255,255)
    isClosed = True
    x = image
    k = keys
    #defining face shape:
    p1 = [[int(k[4]),int(k[5])] , [int(k[12]),int(k[13])] , [int(k[14]),int(k[15])] , [int(k[6]),int(k[7])], [int(k[0]),int(k[1])]]
    p2 = [[int(k[8]),int(k[9])] , [int(k[16]),int(k[17])] , [int(k[18]),int(k[19])] , [int(k[10]),int(k[11])], [int(k[2]),int(k[3])]]
    p3 = [[int(k[20]),int(k[21])], [int(k[4]),int(k[5])] , [int(k[0]),int(k[1])], [int(k[6]),int(k[7])],[int(k[20]),int(k[21])], [int(k[8]),int(k[9])] , [int(k[2]),int(k[3])], [int(k[10]),int(k[11])]]
#     p4 = [[int(k[20]),int(k[21])], [int(k[8]),int(k[9])] , [int(k[2]),int(k[3])], [int(k[10]),int(k[11])]]
    p5 = [[int(k[10]),int(k[11])] , [int(k[20]),int(k[21])] , [int(k[22]),int(k[23])], [int(k[6]),int(k[7])], [int(k[24]),int(k[25])]]
    p6 = [[int(k[22]),int(k[23])], [int(k[26]),int(k[27])] , [int(k[24]),int(k[25])], [int(k[28]),int(k[29])]]
    feature=[p5,p3,p1,p2,p6]
    for i in feature:
        for j in i:
            x=cv2.circle(np.reshape(np.array(x),(h,w,3)).astype('uint8'),(j[0],j[1]), radius = 2 ,color = (0,0,255),thickness = -1)
        x = cv2.polylines(np.reshape(np.array(x),(h,w,3)).astype('uint8'), [np.array(i)], isClosed ,color,thickness = 0)
    return x
    
    


In [5]:
def photo(path):
    img = cv2.imread(path)
    img = cv2.resize(img , (500,500))
    model = get_model()
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        all_x = []
        all_y = []
        final_key = []
        j=0
        crop_image=gray[y:y+h , x:x+h]
        dx= w/96
        dy= h/96
        resize_image = cv2.resize(crop_image, (96,96))
        resize_image = np.array(resize_image).reshape(1,96,96)
        c = np.array(model.predict(resize_image)).reshape(30)
        for i in range(15):
            all_x.append(math.ceil(c[j] * dx) + x)
            all_y.append(math.ceil(c[j+1] * dy) + y)
            j=j+2
        for i in range(15):
            final_key.append(int(all_x[i]))
            final_key.append(int(all_y[i]))
        final_key = np.array(final_key).reshape(30)
        img = key_fit(img , final_key, int(np.array(gray).shape[1]), int(np.array(gray).shape[0]))
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()





In [6]:
def camera():
    face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    model = get_model()
    while True:
        ret ,img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            #keypoint detecting:
            all_x = []
            all_y = []
            final_key = []
            j=0
            crop_image=gray[y:y+h , x:x+h]
            dx= w/96
            dy= h/96
            resize_image = cv2.resize(crop_image, (96,96))
            resize_image = np.array(resize_image).reshape(1,96,96)
            c = np.array(model.predict(resize_image)).reshape(30)
            for i in range(15):
                all_x.append(math.ceil(c[j] * dx) + x)
                all_y.append(math.ceil(c[j+1] * dy) + y)
                j=j+2
            for i in range(15):
                final_key.append(int(all_x[i]))
                final_key.append(int(all_y[i]))
            final_key = np.array(final_key).reshape(30)
            img = key_fit(img , final_key, int(width), int(height))


        cv2.imshow('img',img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
def video(path):
    print("loading the algorithm....")
    face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    model = get_model()
    print("PROCESSING THE VIDEO...please wait")
    while(cap.isOpened()):
        ret ,img = cap.read()
#         img = cv2.resize(img, (700,400))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            #keypoint detecting:
            all_x = []
            all_y = []
            final_key = []
            j=0
            crop_image=gray[y:y+h , x:x+h]
            dx= w/96
            dy= h/96
            resize_image = cv2.resize(crop_image, (96,96))
            resize_image = np.array(resize_image).reshape(1,96,96)
            c = np.array(model.predict(resize_image)).reshape(30)
            for i in range(15):
                all_x.append(math.ceil(c[j] * dx) + x)
                all_y.append(math.ceil(c[j+1] * dy) + y)
                j=j+2
            for i in range(15):
                final_key.append(int(all_x[i]))
                final_key.append(int(all_y[i]))
            final_key = np.array(final_key).reshape(30)
            img = key_fit(img , final_key, int(width), int(height))   

        cv2.imshow('img',img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
print("1) realtime keypoint tracking \n 2) image keypoint tracking \n 3) video keypoint tracking")
choice = input()
if(choice == '1'):
    camera()
elif(choice == '2'):
    print("enter the image path:")
    path = input()
    photo(path)
else:
    

    print("enter the video path:")
    path = input()
    video(path)

In [14]:
### try image_path = 5.jpeg or  9.jpg
#try video_path = original.MOV
# press escape to get out of the live webcam